<!DOCTYPE html>
<html>
<head>
<title>Object Detection and Comparison</title>
</head>
<body>
<h1>Object Detection and Comparison</h1>
<p>This HTML page provides an overview of the Python script that uses the Ultralytics YOLO library for object detection and entity comparison.</p>

<ol>
    <li>Import the necessary libraries: The script starts by importing the Ultralytics YOLO library (<code>ultralytics.YOLO</code>), the operating system module (<code>os</code>), and the CSV module (<code>csv</code>).</li>
    <li>Load YOLO Model: The YOLO model is loaded using the pretrained model file named <code>'yolov8n.pt'</code>.</li>
    <li>Main Function: The <code>main()</code> function is defined as the entry point of the script. It processes subfolders within the main folder and performs comparisons.</li>
    <li>Get Subfolders: The <code>get_subfolders(parent_folder)</code> function retrieves the paths of subfolders within a specified main folder.</li>
    <li>Get Paths of Ideal and Other Images: The <code>get_ideal_images(folder_path)</code> function retrieves paths of ideal images from the "ideal" subfolder, and the <code>get_other_images(folder_path)</code> function retrieves paths of other images from the "other" subfolder.</li>
    <li>Ideal Images Processing: The <code>ideal_images(selected_folder_path)</code> function processes ideal images, predicts entities using the YOLO model, and stores the results.</li>
    <li>Counting Detected Entities and Coordinates: The <code>count_Entities(results)</code> function counts detected entities and extracts their coordinates from the YOLO results.</li>
    <li>Comparing Entities: The <code>compare_entities(ideal_results, other_result)</code> function compares entities detected in ideal and other images, identifying missing and extra entities along with their coordinates.</li>
    <li>Writing Data to CSV: The <code>perform_comparison_and_write_csv(ideal_results, other_path, selected_folder_path)</code> function performs entity comparison, writes results to a CSV file named after the other image, and includes information about missing and extra entities.</li>
    <li>Calling Functions Inside <code>main()</code>: The main function iterates through subfolders, processes images, performs comparisons, and writes results to CSV files.</li>
    <li>Calling Main Function: The main function is executed when the script is run as the main module.</li>
</ol>

<p>This Python script efficiently processes images, performs entity comparison, and writes comparison results to CSV files using the Ultralytics YOLO library.</p>
</body>
</html>


In [ ]:
from ultralytics import YOLO #pip install YOLO
import os
import csv

# Load YOLO model
model = YOLO('yolov8n.pt')  # Using yolov8n pretrained model
folder_path = './Ads_problem4'

def main():
    folders_to_process = get_subfolders(folder_path)
    
    for selected_folder_path in folders_to_process:
        print(f"Processing folder: {selected_folder_path}")
        try:
            ideal_results = ideal_images(selected_folder_path)
            other_images_paths = get_other_images(selected_folder_path)
            
            for other_path in other_images_paths:
                perform_comparison_and_write_csv(ideal_results, other_path, selected_folder_path)
        
        except Exception as e:
            print(f"An error occurred while processing folder '{selected_folder_path}': {str(e)}")


# Function to get subfolders within a directory
def get_subfolders(parent_folder):
    subfolders = []
    for item in os.listdir(parent_folder):
        item_path = os.path.join(parent_folder, item)
        if os.path.isdir(item_path):
            subfolders.append(item_path)
    return subfolders


# Function to get paths of ideal images
def get_ideal_images(folder_path):
    ideal_folder = os.path.join(folder_path, "ideal")
    ideal_images = [os.path.join(ideal_folder, image) for image in os.listdir(ideal_folder)]
    return ideal_images

# Function to get paths of other images
def get_other_images(folder_path):
    other_folder = os.path.join(folder_path, "other")
    other_images = [os.path.join(other_folder, image) for image in os.listdir(other_folder)]
    return other_images



"""
Passing the path through functions for getting prediction data
"""



# Function to process ideal images and predict using the model
def ideal_images(selected_folder_path):
    all_results = []
    # Get paths of images in "ideal" folder
    ideal_images = get_ideal_images(selected_folder_path)
    for path in ideal_images:
        results = model.predict(path, save=True)  # Run YOLO prediction
        all_results.append(results)
    return all_results

# Function to process other images and predict using the model


"""
Functions to count detected entities, cords and comparing them
"""

def count_Entities(results):
    counts = {}
    cords = []

    for result in results:
        boxes = result[0].boxes.cpu().numpy()  # Assuming 'pred' attribute contains boxes
        cords.extend(result[0].boxes.xyxy.cpu().numpy())

    for result in results:
        boxes = result[0].boxes.cpu().numpy()
        for box in boxes:
            cls = int(box.cls[0])
            if cls not in counts:
                counts[cls] = 1
            else:
                counts[cls] += 1
    
    return counts, cords


# Function to compare entities between ideal and other results
def compare_entities(ideal_results, other_result):
    ideal_counts, ideal_cords = count_Entities(ideal_results)
    other_counts, other_cords = count_Entities(other_result)
    
    missing_entities = {}
    extra_entities = {}
    
    # Find missing entities
    for cls, count in ideal_counts.items():
        if cls not in other_counts:
            missing_entities[cls] = count
    
    # Find extra entities
    for cls, count in other_counts.items():
        if cls not in ideal_counts:
            extra_entities[cls] = count
    
    missing_cords = {cls: ideal_cords[0] for cls in missing_entities}
    extra_cords = {cls: other_cords[0] for cls in extra_entities}
    
    return missing_entities, extra_entities, missing_cords, extra_cords



"""
Writing the Data to CSV 
"""
def perform_comparison_and_write_csv(ideal_results, other_path, selected_folder_path):
    other_results = model.predict(other_path, save=True)
    
    # Compare entities between ideal and other images
    missing_entities, extra_entities, missing_cords, extra_cords = compare_entities(ideal_results, other_results)
    
    # Write missing and extra entities to a CSV file with the same name as the other image
    other_image_name = os.path.splitext(os.path.basename(other_path))[0]
    csv_filename = f"{selected_folder_path}/other/{other_image_name}.csv"
    
    with open(csv_filename, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(["Entity", "Difference", "Meta"])
        
        for cls, missing_count in missing_entities.items():
            csv_writer.writerow([f"Entity-class{cls}", "Missing", missing_cords[cls]])
        
        for cls, extra_count in extra_entities.items():
            csv_writer.writerow([f"Entity-class{cls}", "Extra", extra_cords[cls]])
    
    print(f"Comparison results for '{other_image_name}' written to {csv_filename}")



#Calling the Function inside main
if __name__ == "__main__":
    main()
